### ERPA methods: particle-hola RPA, further implementation details

To go with notebook: docs_rpa_particlehole


Eqs 7. in [Tahir 2019](https://doi.org/10.1103/PhysRevB.99.195149) defines the RPA correlation energy , $E^{f-phRPA}_c$, as:  
$$
E^{f-phRPA}_c = \frac{1}{4}(\sum_{w>0} w -Tr(A))
$$

where
$$
A_{ia,jb}=\delta_{ij}\delta_{ab}(\epsilon_a - \epsilon_i) + <aj||ib>
$$

and the RPA equation is:
$$
\begin{bmatrix}
A & B\\
B^* & A^*
\end{bmatrix}
\begin{pmatrix}
X\\
Y
\end{pmatrix}
= \omega_n
\begin{bmatrix}
1 & 0\\
0 & -1
\end{bmatrix}
\begin{pmatrix}
X\\
Y
\end{pmatrix}
$$

[Scuseria et al.](http://aip.scitation.org/doi/10.1063/1.4820557) define instead:
$$
E^{phRPA}_c = \frac{1}{2}(\sum_{w>0} w -Tr(A))
$$

Therefore, Scuseria's and Tahir's p-h RPA correlation energy differ by a factor of two. 

# PySCF RPA

The key elements terms in the correlation energy equations above $A$ and $\omega$, are evaluated with PySCF, so I describe bellow how PySCF defines them as this may be one source of discrepancies.

RPA equation solved by PySCF:

$$
\begin{bmatrix}
A & B\\
-B^* & -A^*
\end{bmatrix}
\begin{pmatrix}
X\\
Y
\end{pmatrix}
= \omega_n
\begin{bmatrix}
1 & 0\\
0 & 1
\end{bmatrix}
\begin{pmatrix}
X\\
Y
\end{pmatrix}
$$

where the matrix $A$ has the following spin structure (same for $B$):

$$
\begin{bmatrix}
A_{aa,aa} & A_{aa,bb}\\
A^T_{aa,bb} & A_{bb,bb}
\end{bmatrix}
$$
(here the subscripts `a` and `b` stand for $\alpha$ and $\beta$ spin-orbitals)

and each element is evaluated as:
$$
A_{ia,jb}=\delta_{ij}\delta_{ab}(\epsilon_a - \epsilon_i) + (ia||bj)
$$
where the indexes `i,j` correspond to occupied orbitlas and `a,b` to virtuals.

(Notice PySCF uses chemist notation for the two-electron integrals while Tahir uses the physicist one; still, the value of corresponding terms will be the same.)


After getting the necessary inputs from PySCF, I evaluete the ph-RPA correlation energy as:

In [ ]:
def ecorr(w, A_aa, A_bb, factor=None):
    """Particle-hole RPA correlation energy

    E^{f-phRPA}_c = factor*(\sum_{w>0} w -Tr(A))
    
    A_{ia,jb}=\delta_{ij}\delta_{ab}(\epsilon_a - \epsilon_i) + (ia||bj)

    Here, A_{aa}=A_{aa,aa} and  A_{bb}=A_{bb,bb}

    factor: 1 if None (default), 0.25 if `t` (Thahir's equation), 0.5 if `s` (Scuseria's equation)

    """
    w[w < 0.] = 0.0
    ecorr = sum(w.real) - numpy.trace(A_aa) - numpy.trace(A_bb)
    if factor is None:
        cnt = 1
    elif factor == 't':
        cnt = 0.25
    elif factor == 's':
        cnt = 0.5
    return cnt * ecorr